In [8]:
# Import necessary libraries
import pandas as pd
import os
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from sklearn.linear_model import LinearRegression
from alpaca_trade_api.rest import REST, APIError
import configparser
import holidays
import pytz
import time
import os
import warnings
import traceback
from tqdm.notebook import tqdm
from datetime import datetime, timedelta



In [9]:

def backtest_pair(price_data, stock1, stock2, thresholds, initial_capital=100000, 
                  position_size=0.1, commission=0.001):
    """
    Enhanced backtester for pairs trading
    
    Parameters:
    -----------
    price_data : dict
        Dictionary containing price data for each stock
    stock1, stock2 : str
        Stock symbols to trade
    thresholds : dict
        Dictionary containing entry_threshold, exit_threshold, and stop_loss
    initial_capital : float
        Starting capital
    position_size : float
        Percentage of capital to allocate per trade (0.1 = 10%)
    commission : float
        Commission per trade as percentage (0.001 = 0.1%)
    """
    # Extract and format price data
    s1 = np.array(price_data[stock1], dtype=float).flatten()
    s2 = np.array(price_data[stock2], dtype=float).flatten()
    
    # Make sure we have dates for both stocks
    dates = pd.date_range(start='2023-01-01', periods=len(s1))  # Assume dates if none available
    if 'dates' in price_data:
        dates = price_data['dates'][-len(s1):]
    
    # Calculate minimum length and align
    min_len = min(len(s1), len(s2))
    s1, s2 = s1[-min_len:], s2[-min_len:]
    dates = dates[-min_len:]
    
    # Calculate hedge ratio
    X = sm.add_constant(s1)
    model = sm.OLS(s2, X).fit()
    hedge_ratio = float(model.params[1])
    intercept = float(model.params[0])
    
    # Calculate spread and z-scores
    spread = s2 - (hedge_ratio * s1 + intercept)
    rolling_mean = pd.Series(spread).rolling(window=20).mean().values
    rolling_std = pd.Series(spread).rolling(window=20).std().values
    
    # Handle initial NaN values
    rolling_mean[:20] = spread[:20].mean()
    rolling_std[:20] = spread[:20].std()
    
    # Calculate z-scores
    zscores = (spread - rolling_mean) / rolling_std
    
    # Initialize tracking variables
    capital = initial_capital
    in_position = False
    position_type = None
    entry_capital = 0
    
    # Performance tracking
    daily_returns = []
    positions = []
    trades = []
    capital_history = [initial_capital]
    
    # Main trading loop
    for i in range(1, len(spread)):
        z = zscores[i]
        z_prev = zscores[i-1]
        s1_price = s1[i]
        s2_price = s2[i]
        current_date = dates[i]
        
        # Update capital history
        if not in_position:
            capital_history.append(capital)
        else:
            # Calculate current position value
            if position_type == "long":
                # Long spread: long s2, short s1
                pos_value = (s2_price - entry_s2) - hedge_ratio * (s1_price - entry_s1)
                current_capital = entry_capital + (pos_size * pos_value)
            else:
                # Short spread: short s2, long s1
                pos_value = (entry_s2 - s2_price) - hedge_ratio * (entry_s1 - s1_price)
                current_capital = entry_capital + (pos_size * pos_value)
            
            capital_history.append(current_capital)
            
        # Trading logic
        if not in_position:
            # Check for entry signals
            if z > thresholds['entry_threshold'] and z_prev <= thresholds['entry_threshold']:
                # Short the spread: short s2, long s1
                in_position = True
                position_type = "short"
                entry_s1 = s1_price
                entry_s2 = s2_price
                entry_z = z
                entry_index = i
                entry_date = current_date
                
                # Position sizing
                pos_size = capital * position_size
                entry_capital = capital
                
                # Record trade
                positions.append({
                    'type': 'short',
                    'entry_date': entry_date,
                    'entry_z': entry_z,
                    'stock1_price': entry_s1,
                    'stock2_price': entry_s2
                })
                
                # Apply commission costs
                capital -= (pos_size * commission * 2)  # Commission for both legs
                
            elif z < -thresholds['entry_threshold'] and z_prev >= -thresholds['entry_threshold']:
                # Long the spread: long s2, short s1
                in_position = True
                position_type = "long"
                entry_s1 = s1_price
                entry_s2 = s2_price
                entry_z = z
                entry_index = i
                entry_date = current_date
                
                # Position sizing
                pos_size = capital * position_size
                entry_capital = capital
                
                # Record trade
                positions.append({
                    'type': 'long',
                    'entry_date': entry_date,
                    'entry_z': entry_z,
                    'stock1_price': entry_s1,
                    'stock2_price': entry_s2
                })
                
                # Apply commission costs
                capital -= (pos_size * commission * 2)  # Commission for both legs
                
        else:
            # Check for exit signals
            exit_signal = False
            exit_reason = None
            
            # Exit based on z-score crossing back
            if (position_type == "long" and z >= -thresholds['exit_threshold'] and z_prev < -thresholds['exit_threshold']):
                exit_signal = True
                exit_reason = "target"
            elif (position_type == "short" and z <= thresholds['exit_threshold'] and z_prev > thresholds['exit_threshold']):
                exit_signal = True
                exit_reason = "target"
                
            # Stop loss - z-score moves further from equilibrium
            elif (position_type == "long" and z < entry_z - thresholds['stop_loss']):
                exit_signal = True
                exit_reason = "stop_loss"
            elif (position_type == "short" and z > entry_z + thresholds['stop_loss']):
                exit_signal = True
                exit_reason = "stop_loss"
                
            # Time-based exit (optional) - exit after 20 days
            elif i - entry_index > 20:
                exit_signal = True
                exit_reason = "time_exit"
                
            if exit_signal:
                # Calculate profit
                if position_type == "long":
                    profit = (s2_price - entry_s2) - hedge_ratio * (s1_price - entry_s1)
                else:
                    profit = (entry_s2 - s2_price) - hedge_ratio * (entry_s1 - s1_price)
                
                # Apply position sizing and commission
                trade_pnl = pos_size * profit - (pos_size * commission * 2)
                capital = entry_capital + trade_pnl
                
                # Record trade
                trades.append({
                    'type': position_type,
                    'entry_date': entry_date,
                    'exit_date': current_date,
                    'holding_days': i - entry_index,
                    'entry_z': entry_z,
                    'exit_z': z,
                    'pnl': trade_pnl,
                    'return': trade_pnl / entry_capital * 100,
                    'exit_reason': exit_reason
                })
                
                in_position = False
    
    # Close any open position at the end
    if in_position:
        if position_type == "long":
            profit = (s2[-1] - entry_s2) - hedge_ratio * (s1[-1] - entry_s1)
        else:
            profit = (entry_s2 - s2[-1]) - hedge_ratio * (entry_s1 - s1[-1])
        
        trade_pnl = pos_size * profit - (pos_size * commission * 2)
        capital = entry_capital + trade_pnl
        
        trades.append({
            'type': position_type,
            'entry_date': entry_date,
            'exit_date': dates[-1],
            'holding_days': len(spread) - entry_index,
            'entry_z': entry_z,
            'exit_z': zscores[-1],
            'pnl': trade_pnl,
            'return': trade_pnl / entry_capital * 100,
            'exit_reason': 'end_of_period'
        })
    
    # Calculate performance metrics
    capital_history = np.array(capital_history)
    returns = np.diff(capital_history) / capital_history[:-1]
    
    # Key performance metrics
    total_return = (capital - initial_capital) / initial_capital * 100
    num_trades = len(trades)
    win_trades = sum(1 for t in trades if t['pnl'] > 0)
    win_rate = win_trades / num_trades if num_trades > 0 else 0
    
    avg_return = np.mean([t['return'] for t in trades]) if trades else 0
    avg_hold_days = np.mean([t['holding_days'] for t in trades]) if trades else 0
    
    # Sharpe ratio (annualized)
    daily_ret = pd.Series(returns).fillna(0)
    sharpe = np.sqrt(252) * daily_ret.mean() / daily_ret.std() if daily_ret.std() > 0 else 0
    
    # Max drawdown
    peak = np.maximum.accumulate(capital_history)
    drawdown = (peak - capital_history) / peak
    max_drawdown = drawdown.max() * 100
    
    # Plots
    plt.figure(figsize=(14, 10))
    
    # Z-score plot
    plt.subplot(3, 1, 1)
    plt.plot(zscores, label='Z-Score')
    plt.axhline(thresholds['entry_threshold'], color='r', linestyle='--', label='Entry')
    plt.axhline(-thresholds['entry_threshold'], color='r', linestyle='--')
    plt.axhline(thresholds['exit_threshold'], color='g', linestyle='--', label='Exit')
    plt.axhline(-thresholds['exit_threshold'], color='g', linestyle='--')
    plt.axhline(thresholds['stop_loss'], color='y', linestyle='--', label='Stop Loss')
    plt.axhline(-thresholds['stop_loss'], color='y', linestyle='--')
    
    # Mark entry and exit points
    for t in trades:
        entry_idx = np.where(dates == t['entry_date'])[0][0]
        exit_idx = np.where(dates == t['exit_date'])[0][0]
        
        if t['type'] == 'long':
            plt.scatter(entry_idx, zscores[entry_idx], color='g', marker='^', s=100)
            plt.scatter(exit_idx, zscores[exit_idx], color='r', marker='v', s=100)
        else:
            plt.scatter(entry_idx, zscores[entry_idx], color='r', marker='v', s=100)
            plt.scatter(exit_idx, zscores[exit_idx], color='g', marker='^', s=100)
    
    plt.title(f"Z-Score for {stock1}-{stock2}")
    plt.legend()
    plt.grid(True)
    
    # Capital history
    plt.subplot(3, 1, 2)
    plt.plot(capital_history, label='Portfolio Value')
    plt.title("Portfolio Value Over Time")
    plt.grid(True)
    plt.legend()
    
    # Drawdown chart
    plt.subplot(3, 1, 3)
    plt.fill_between(range(len(drawdown)), 0, drawdown * 100)
    plt.title(f"Drawdown (%) - Max: {max_drawdown:.2f}%")
    plt.grid(True)
    
    plt.tight_layout()
    
    # Summary table
    summary = {
        'Pair': f"{stock1}-{stock2}",
        'Hedge Ratio': hedge_ratio,
        'Initial Capital': initial_capital,
        'Final Capital': capital,
        'Total Return (%)': total_return,
        'Number of Trades': num_trades,
        'Win Rate (%)': win_rate * 100,
        'Avg Trade Return (%)': avg_return,
        'Avg Holding Period (days)': avg_hold_days,
        'Sharpe Ratio': sharpe,
        'Max Drawdown (%)': max_drawdown
    }
    
    return {
        'summary': summary,
        'trades': trades,
        'capital_history': capital_history,
        'zscores': zscores,
        'hedge_ratio': hedge_ratio
    }

In [15]:



class PairsTrader:
    """
    A class to implement a statistical arbitrage pairs trading strategy.
    """

    def __init__(self, api, lookback_period=60, z_score_threshold=2.0,
                 max_active_pairs=5, position_size=0.1):
        """
        Initialize the pairs trading strategy.

        Parameters:
        -----------
        api : alpaca_trade_api.REST
            The Alpaca API client
        lookback_period : int
            Number of days to use for cointegration analysis
        z_score_threshold : float
            Z-score threshold to trigger trades
        max_active_pairs : int
            Maximum number of pairs to trade simultaneously
        position_size : float
            Percentage of portfolio value to allocate per pair
        """
        self.api = api
        self.lookback_period = lookback_period
        self.z_score_threshold = z_score_threshold
        self.max_active_pairs = max_active_pairs
        self.position_size = position_size
        self.active_pairs = {}  # Store active pairs and their details
        self.pair_stats = {}  # Store pair statistics
        self.adaptive_thresholds = {}

        # Create directory for logs and results
        os.makedirs("pairs_logs", exist_ok=True)

        # Initialize log file
        self.log_file = f"pairs_logs/pairs_trading_{datetime.now().strftime('%Y%m%d')}.log"
        with open(self.log_file, "a") as f:
            f.write(f"Pairs Trading Bot Initialized at {datetime.now()}\n")
            f.write(f"Parameters: Lookback={lookback_period}, Z-Score={z_score_threshold}, " +
                    f"Max Pairs={max_active_pairs}, Position Size={position_size * 100}%\n\n")

    def log(self, message):
        """Log a message to both console and file."""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_message = f"[{timestamp}] {message}"
        print(log_message)
        with open(self.log_file, "a") as f:
            f.write(log_message + "\n")

    def get_stock_sectors(self, tickers):
        """
        Get sector and industry information for a list of stock tickers using Yahoo Finance
        
        Parameters:
        -----------
        tickers : list
            List of stock tickers
            
        Returns:
        --------
        DataFrame
            DataFrame with ticker, sector, and industry information
        """
        sector_data = []
        
        # Process tickers in batches to avoid rate limiting
        batch_size = 20
        for i in range(0, len(tickers), batch_size):
            batch = tickers[i:i+batch_size]
            print(f"Processing batch {i//batch_size + 1}/{(len(tickers)-1)//batch_size + 1}...")
            
            for ticker in tqdm(batch):
                try:
                    # Get stock info
                    stock = yf.Ticker(ticker)
                    info = stock.info
                    
                    # Extract sector and industry info
                    sector = info.get('sector', 'Unknown')
                    industry = info.get('industry', 'Unknown')
                    
                    sector_data.append({
                        'ticker': ticker,
                        'sector': sector,
                        'industry': industry
                    })
                    
                except Exception as e:
                    print(f"Error getting info for {ticker}: {e}")
                    sector_data.append({
                        'ticker': ticker,
                        'sector': 'Unknown',
                        'industry': 'Unknown'
                    })
        
        # Create DataFrame
        sector_df = pd.DataFrame(sector_data)
        
        return sector_df
    
    # Function to group stocks by sector and industry
    def group_by_sector_industry(self, sector_df):
        """
        Group stocks by sector and industry
        
        Parameters:
        -----------
        sector_df : DataFrame
            DataFrame with ticker, sector, and industry information
            
        Returns:
        --------
        dict
            Dictionary with sector and industry groupings
        """
        # Group by sector
        sector_groups = {}
        for sector in sector_df['sector'].unique():
            if sector != 'Unknown':
                sector_stocks = sector_df[sector_df['sector'] == sector]['ticker'].tolist()
                sector_groups[sector] = sector_stocks
        
        # Group by industry
        industry_groups = {}
        for industry in sector_df['industry'].unique():
            if industry != 'Unknown':
                industry_stocks = sector_df[sector_df['industry'] == industry]['ticker'].tolist()
                industry_groups[industry] = industry_stocks
        
        return {
            'sector': sector_groups,
            'industry': industry_groups
        }
    
    # Function to find cointegrated pairs within sector/industry groups
    def find_sector_based_pairs(self, price_data, groupings, lookback_period=60, p_value_threshold=0.05):
        """
        Find cointegrated pairs within the same sector or industry
        
        Parameters:
        -----------
        price_data : dict
            Dictionary with stock prices by ticker
        groupings : dict
            Dictionary with sector and industry groupings
        lookback_period : int
            Number of days to use for cointegration analysis
        p_value_threshold : float
            P-value threshold for cointegration test
            
        Returns:
        --------
        list
            List of tuples (stock1, stock2, p_value, group_type, group_name) sorted by p_value
        """
        pairs = []
        
        # Process industry groups first (more specific)
        for industry, stocks in groupings['industry'].items():
            if len(stocks) < 2:
                continue
                
            print(f"Processing industry: {industry} ({len(stocks)} stocks)")
            # Find pairs within this industry
            industry_pairs = self._find_pairs_in_group(price_data, stocks, lookback_period, p_value_threshold)
            
            # Add industry information to pairs
            for stock1, stock2, p_value in industry_pairs:
                pairs.append((stock1, stock2, p_value, 'industry', industry))
        
        # Process sector groups
        for sector, stocks in groupings['sector'].items():
            if len(stocks) < 2:
                continue
                
            print(f"Processing sector: {sector} ({len(stocks)} stocks)")
            # Find pairs within this sector
            sector_pairs = self._find_pairs_in_group(price_data, stocks, lookback_period, p_value_threshold)
            
            # Add sector information to pairs
            for stock1, stock2, p_value in sector_pairs:
                # Check if this pair is already included from industry analysis
                if not any((p[0] == stock1 and p[1] == stock2) or (p[0] == stock2 and p[1] == stock1) for p in pairs):
                    pairs.append((stock1, stock2, p_value, 'sector', sector))
        
        # Sort by p-value
        pairs.sort(key=lambda x: x[2])
        
        return pairs
    
    def _find_pairs_in_group(self, price_data, stocks, lookback_period, p_value_threshold):
        """Helper function to find cointegrated pairs within a group of stocks"""
        group_pairs = []
        
        # Filter stocks that have price data
        valid_stocks = [s for s in stocks if s in price_data]
        
        # Find cointegrated pairs
        for i, stock1 in enumerate(valid_stocks):
            for stock2 in valid_stocks[i+1:]:
                try:
                    # Make sure data lengths match
                    min_len = min(len(price_data[stock1]), len(price_data[stock2]))
                    
                    # Skip if not enough data
                    if min_len < lookback_period * 0.6:
                        continue
                        
                    stock1_prices = price_data[stock1][-min_len:]
                    stock2_prices = price_data[stock2][-min_len:]
                    
                    # Perform cointegration test
                    score, p_value, _ = coint(stock1_prices, stock2_prices)
                    
                    if p_value < p_value_threshold:  # Significant cointegration
                        group_pairs.append((stock1, stock2, p_value))
                except Exception as e:
                    continue
        
        return group_pairs
    
    # Example function to visualize price series and spread of a pair
    def visualize_pair(self, stock1, stock2, price_data, hedge_ratio=None):
        """
        Visualize price series and spread of a stock pair
        
        Parameters:
        -----------
        stock1, stock2 : str
            Stock symbols
        price_data : dict
            Dictionary with price data
        hedge_ratio : float or None
            Hedge ratio for calculating spread. If None, will be calculated
        """

        plot_dir = "pair_visualizations"
        os.makedirs(plot_dir, exist_ok=True)
        
        # Get price data
        s1 = price_data[stock1]
        s2 = price_data[stock2]
        
        # Make sure lengths match
        min_len = min(len(s1), len(s2))
        s1 = s1[-min_len:]
        s2 = s2[-min_len:]
        
        # Calculate hedge ratio if not provided
        if hedge_ratio is None:
            # Simple linear regression
            hedge_ratio = np.polyfit(s1, s2, 1)[0]
        
        # Calculate spread
        spread = s2 - hedge_ratio * s1
        
        # Normalize price series for visualization
        s1_norm = s1 / s1[0]
        s2_norm = s2 / s2[0]
        
        # Create figure with 2 subplots
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
        
        # Plot normalized price series
        ax1.plot(s1_norm, label=stock1)
        ax1.plot(s2_norm, label=stock2)
        ax1.set_title(f"Normalized Price Series: {stock1} vs {stock2}")
        ax1.legend()
        ax1.grid(True)
        
        # Plot spread
        ax2.plot(spread)
        ax2.set_title(f"Spread (Hedge Ratio: {hedge_ratio:.4f})")
        ax2.axhline(y=np.mean(spread), color='r', linestyle='--', label='Mean')
        ax2.axhline(y=np.mean(spread) + np.std(spread), color='g', linestyle='--', label='+1 STD')
        ax2.axhline(y=np.mean(spread) - np.std(spread), color='g', linestyle='--', label='-1 STD')
        ax2.axhline(y=np.mean(spread) + 2*np.std(spread), color='y', linestyle='--', label='+2 STD')
        ax2.axhline(y=np.mean(spread) - 2*np.std(spread), color='y', linestyle='--', label='-2 STD')
        ax2.grid(True)
        ax2.legend()
        
        plt.tight_layout()
        
        
        plot_filename = f"{stock1}_{stock2}_visualization.png" # Creates filenames like "AAPL_MSFT_visualization.png"
        plot_filepath = os.path.join(plot_dir, plot_filename)
        try:
            plt.savefig(plot_filepath)
            self.log(f"Saved plot for {stock1}-{stock2} to {plot_filepath}") # Optional: log saving action
        except Exception as e:
            self.log(f"Error saving plot for {stock1}-{stock2}: {e}") # Optional: log errors
        
        # --- Close the figure to free memory ---
        plt.close(fig) # Add this line immediately after saving
                
        # Stationarity test results
        mean = np.mean(spread)
        std = np.std(spread)
        current = float(spread[-1])
        z_score = (current - mean) / std
        
        print(f"Spread Mean: {mean:.4f}")
        print(f"Spread Std Dev: {std:.4f}")
        print(f"Current Spread: {current:.4f}")
        print(f"Current Z-Score: {z_score:.4f}")
        
        return spread

     # Function to calculate adaptive Z-score thresholds based on historical behavior
    def calculate_adaptive_thresholds(self, price_data, stock1, stock2, lookback_period=60, 
                                      min_threshold=1.5, max_threshold=3.0):
        """
        Calculate adaptive Z-score thresholds based on historical behavior of the pair
        
        Parameters:
        -----------
        price_data : dict
            Dictionary with price data for all stocks
        stock1, stock2 : str
            Stock symbols for the pair
        lookback_period : int
            Number of days to use for analysis
        min_threshold : float
            Minimum Z-score threshold to use
        max_threshold : float
            Maximum Z-score threshold to use
            
        Returns:
        --------
        dict
            Dictionary with entry and exit thresholds
        """
        # Get price data
        s1 = price_data[stock1]
        s2 = price_data[stock2]
    
        # Ensure data is in the correct format (1D numeric arrays)
        try:
            s1 = np.array(s1, dtype=float).flatten()
            s2 = np.array(s2, dtype=float).flatten()
        except Exception as e:
            self.log(f"Error converting price data to numpy arrays: {e}")
            # Return default thresholds if conversion fails
            return {
                'entry_threshold': 2.0,
                'exit_threshold': 0.5,
                'stop_loss': 3.0
            }
        # Make sure lengths match
        min_len = min(len(s1), len(s2))
        s1 = s1[-min_len:]
        s2 = s2[-min_len:]
        

        
        # Calculate hedge ratio using OLS regression
        X = np.array(s1).reshape(-1, 1)
        X = np.concatenate([np.ones((X.shape[0], 1)), X], axis=1)
        beta = np.linalg.lstsq(X, s2, rcond=None)[0]
        hedge_ratio = beta[1]
        
        # Calculate spread
        spread = s2 - hedge_ratio * s1
        
        # Calculate rolling Z-scores with different windows to capture various time frames
        windows = [5, 10, 20]
        z_scores = []
        
        for window in windows:
            if len(spread) >= window * 2:  # Make sure we have enough data
                rolling_mean = np.convolve(spread, np.ones(window)/window, mode='valid')
                rolling_std = np.array([np.std(spread[i:i+window]) for i in range(len(spread)-window+1)])
                
                # Calculate Z-scores for each point
                roll_z = (spread[window-1:] - rolling_mean) / rolling_std
                z_scores.extend(abs(roll_z))
        
        if not z_scores:
            # Default thresholds if we can't calculate
            return {
                'entry_threshold': 2.0,
                'exit_threshold': 0.5,
                'stop_loss': 3.0
            }
        
        # Find the 90th percentile of historical absolute Z-scores for entry threshold
        entry_threshold = min(max(np.percentile(z_scores, 90), min_threshold), max_threshold)
        
        # Exit threshold typically 0-0.5 standard deviations
        exit_threshold = min(entry_threshold / 4, 0.5)
        
        # Stop loss threshold
        stop_loss = min(entry_threshold * 1.5, max_threshold)
        
        return {
            'entry_threshold': entry_threshold,
            'exit_threshold': exit_threshold,
            'stop_loss': stop_loss,
            'hedge_ratio': hedge_ratio
        }
    
    # Function to apply adaptive thresholds to a group of pairs
    def calculate_all_pair_thresholds(self, pairs, price_data, lookback_period=60):
        """
        Calculate adaptive thresholds for all pairs
        
        Parameters:
        -----------
        pairs : list
            List of pairs (tuples with stock1, stock2, etc.)
        price_data : dict
            Dictionary with price data
        lookback_period : int
            Number of days for lookback
            
        Returns:
        --------
        dict
            Dictionary with thresholds for each pair
        """
        pair_thresholds = {}
        
        for pair_info in pairs:
            stock1, stock2 = pair_info[0], pair_info[1]
            pair_key = (stock1, stock2)
            
            try:
                thresholds = self.calculate_adaptive_thresholds(
                    price_data, stock1, stock2, lookback_period
                )
                pair_thresholds[pair_key] = thresholds
                
                print(f"Pair {stock1}-{stock2}: Entry={thresholds['entry_threshold']:.2f}, "
                      f"Exit={thresholds['exit_threshold']:.2f}, "
                      f"Stop Loss={thresholds['stop_loss']:.2f}, "
                      f"Hedge Ratio={thresholds['hedge_ratio']:.4f}")
                
            except Exception as e:
                print(f"Error calculating thresholds for {stock1}-{stock2}: {e}")

        
        return pair_thresholds

    def find_pairs(self, universe, top_n=20, use_sectors=True):
        """
        Find cointegrated pairs from a universe of stocks, optionally using sector analysis.
    
        Parameters:
        -----------
        universe : list
            List of stock symbols to analyze
        top_n : int
            Number of top pairs to select
        use_sectors : bool
            Whether to use sector/industry analysis to find pairs
    
        Returns:
        --------
        list
            List of tuples (stock1, stock2, p_value) sorted by p_value
        """
        self.log(f"Finding cointegrated pairs from {len(universe)} stocks...")
    
        # Get historical data for all stocks
        price_data = {}
        for symbol in universe:
            try:
                # Get data from Yahoo Finance
                start_date = (datetime.now() - timedelta(days=self.lookback_period)).strftime("%Y-%m-%d")
                end_date = datetime.now().strftime("%Y-%m-%d")
                data = yf.download(symbol, start=start_date, end=end_date, progress=False)
                
                if len(data) >= self.lookback_period * 0.6:  # Require at least 60% data coverage
                    price_data[symbol] = data['Close'].values
                else:
                    self.log(f"Skipped {symbol} since data len ={len(data)} and data coverage = {self.lookback_period * 0.6}")
            except Exception as e:
                self.log(f"Error fetching data for {symbol}: {e}")
        
        self.log(f"Available stocks in price_data: {list(price_data.keys())}")
        
        # Use sector-based analysis if requested
        if use_sectors:
            self.log("Using sector/industry analysis to find pairs...")
            # Get sector/industry information
            sector_df = self.get_stock_sectors(list(price_data.keys()))
            
            # Group stocks by sector and industry
            groupings = self.group_by_sector_industry(sector_df)
            
            # Find pairs within sectors/industries
            pairs = self.find_sector_based_pairs(
                price_data, 
                groupings, 
                lookback_period=self.lookback_period, 
                p_value_threshold=0.05
            )
            
            # Format pairs to match expected return format
            formatted_pairs = [(p[0], p[1], p[2]) for p in pairs]
            
            # Log sector/industry information
            for p in pairs[:top_n]:
                stock1, stock2, p_value, group_type, group_name = p
                self.log(f"Found pair {stock1}-{stock2} with p-value {p_value:.6f} in {group_type}: {group_name}")
                
        else:
            # Original implementation for finding pairs
            pairs = []
            for i, stock1 in enumerate(price_data.keys()):
                for stock2 in list(price_data.keys())[i + 1:]:
                    # Skip if either stock doesn't have enough data
                    if len(price_data[stock1]) < self.lookback_period * 0.6 or \
                            len(price_data[stock2]) < self.lookback_period * 0.6:
                        continue
    
                    # Make sure data lengths match
                    min_len = min(len(price_data[stock1]), len(price_data[stock2]))
                    stock1_prices = price_data[stock1][-min_len:]
                    stock2_prices = price_data[stock2][-min_len:]
    
                    # Perform cointegration test
                    score, p_value, _ = coint(stock1_prices, stock2_prices)
    
                    if p_value < 0.05:  # 5% significance level
                        pairs.append((stock1, stock2, p_value))
            
            formatted_pairs = pairs
    
        # Sort by p-value (lower is better for cointegration)
        formatted_pairs.sort(key=lambda x: x[2])
    
        # Take top N pairs
        top_pairs = formatted_pairs[:top_n]
        self.log(f"Found {len(top_pairs)} cointegrated pairs out of {len(formatted_pairs)} tested pairs.")
    
        # Calculate and store pair statistics
        for stock1, stock2, p_value in top_pairs:
            self._calculate_pair_stats(stock1, stock2)
    
        for stock1, stock2, p_value in top_pairs:
            hedge_ratio = self.pair_stats.get((stock1, stock2), {}).get('hedge_ratio')
            self.visualize_pair(stock1, stock2, price_data, hedge_ratio=hedge_ratio)
    
        # Calculate adaptive thresholds
        self.adaptive_thresholds = self.calculate_all_pair_thresholds(top_pairs, price_data, self.lookback_period)
        print(f"adaptive_thresholds: {self.adaptive_thresholds}")

        
        return top_pairs, price_data


    def _calculate_pair_stats(self, stock1, stock2):
        try:
            # Get historical data
            start_date = (datetime.now() - timedelta(days=self.lookback_period)).strftime("%Y-%m-%d")
            end_date = datetime.now().strftime("%Y-%m-%d")
    
            stock1_data = yf.download(stock1, start=start_date, end=end_date, progress=False)['Close'].dropna()
            if isinstance(stock1_data, pd.DataFrame):
                stock1_data = stock1_data.squeeze()
            
            stock2_data = yf.download(stock2, start=start_date, end=end_date, progress=False)['Close'].dropna()
            if isinstance(stock2_data, pd.DataFrame):
                stock2_data = stock2_data.squeeze()

            if stock1_data.empty or stock2_data.empty:
                self.log(f"Insufficient data for {stock1}-{stock2}. Skipping...")
                return
    
            # Align lengths
            min_len = min(len(stock1_data), len(stock2_data))
            stock1_data = stock1_data[-min_len:]
            stock2_data = stock2_data[-min_len:]
    
            # Linear regression for hedge ratio
            X = sm.add_constant(stock1_data.values.reshape(-1, 1))
            model = sm.OLS(stock2_data.values, X).fit()
            hedge_ratio = float(model.params[1])
    
            spread = pd.Series(
                stock2_data.values - hedge_ratio * stock1_data.values,
                index=stock1_data.index
            )

    
            if spread.isna().any():
                self.log(f"Spread contains NaN values for pair {stock1}-{stock2}. Skipping...")
                return
    
            mean_spread = spread.mean()
            std_spread = spread.std()
            current_spread = spread.iloc[-1]
    
            if pd.isna(mean_spread) or pd.isna(std_spread) or pd.isna(current_spread):
                self.log(f"Spread stats contain NaN for {stock1}-{stock2}. Skipping...")
                return
    
            if float(std_spread) == 0.0:
                self.log(f"Standard deviation of spread is zero for {stock1}-{stock2}. Skipping...")
                return
    
            z_score = float((current_spread - mean_spread) / std_spread)
    
            self.pair_stats[(stock1, stock2)] = {
                'hedge_ratio': hedge_ratio,
                'mean_spread': mean_spread,
                'std_spread': std_spread,
                'current_spread': current_spread,
                'z_score': z_score
            }
    
           
            self.log(f"Pair {stock1}-{stock2}: Hedge Ratio = {hedge_ratio:.4f}, Z-Score = {z_score:.4f}")
    
        except Exception as e:
            self.log(f"Error calculating stats for pair {stock1}-{stock2}: {e}")
            self.log(traceback.format_exc())


   

            
    def check_for_trading_signals(self):
        """
        Check all monitored pairs for trading signals.
        """
        if not self.pair_stats:
            self.log("No pairs to check for signals.")
            return

        for pair, stats in self.pair_stats.items():
            stock1, stock2 = pair
            z_score = stats['z_score']

            # Update the spread and z-score
            self._update_pair_zscore(stock1, stock2)
            z_score = self.pair_stats[pair]['z_score']

            # Check if pair is already being traded
            if pair in self.active_pairs:
                self._manage_existing_position(stock1, stock2, z_score)
            else:
                self._check_for_new_position(stock1, stock2, z_score)

    def _update_pair_zscore(self, stock1, stock2):
        """
        Update the z-score for a pair.

        Parameters:
        -----------
        stock1, stock2 : str
            Stock symbols
        """
        try:
            # Get latest prices
            stock1_price = float(self.api.get_latest_trade(stock1).price)
            stock2_price = float(self.api.get_latest_trade(stock2).price)

            # Use stored stats to calculate current z-score
            stats = self.pair_stats[(stock1, stock2)]
            hedge_ratio = stats['hedge_ratio']
            mean_spread = stats['mean_spread']
            std_spread = stats['std_spread']

            # Calculate current spread and z-score
            current_spread = stock2_price - hedge_ratio * stock1_price
            z_score = (current_spread - mean_spread) / std_spread

            # Update stats
            self.pair_stats[(stock1, stock2)]['current_spread'] = current_spread
            self.pair_stats[(stock1, stock2)]['z_score'] = z_score

        except Exception as e:
            self.log(f"Error updating z-score for pair {stock1}-{stock2}: {e}")

    def _manage_existing_position(self, stock1, stock2, z_score):
        """
        Manage an existing pairs position.

        Parameters:
        -----------
        stock1, stock2 : str
            Stock symbols
        z_score : float
            Current z-score of the spread
        """
        position_details = self.active_pairs[(stock1, stock2)]
        position_type = position_details['position_type']
        entry_zscore = position_details['entry_zscore']
        thresholds = self.adaptive_thresholds.get((stock1, stock2), {})
        exit_threshold = thresholds.get('exit_threshold', 0.5)


        # Check for exit conditions
        if (position_type == 'long' and z_score >= -exit_threshold) or \
           (position_type == 'short' and z_score <= exit_threshold):
            self._close_position(stock1, stock2, z_score)


        # Check for stop loss (z-score moved too far in wrong direction)
        elif (position_type == 'long' and z_score < entry_zscore - 1.0) or \
                (position_type == 'short' and z_score > entry_zscore + 1.0):
            self.log(
                f"Stop loss triggered for {stock1}-{stock2}. Entry Z-Score: {entry_zscore:.4f}, Current Z-Score: {z_score:.4f}")
            self._close_position(stock1, stock2, z_score)

    def _check_for_new_position(self, stock1, stock2, z_score):
        """
        Check if a new position should be opened.

        Parameters:
        -----------
        stock1, stock2 : str
            Stock symbols
        z_score : float
            Current z-score of the spread
        """

        thresholds = self.adaptive_thresholds.get((stock1, stock2), {})
        entry_threshold = thresholds.get('entry_threshold', self.z_score_threshold)

        # Limit number of active pairs
        if len(self.active_pairs) >= self.max_active_pairs:
            return

        # Check for entry conditions
        if z_score <= -entry_threshold:
            # Long the spread (buy stock2, sell stock1)
            self._open_position(stock1, stock2, z_score, position_type='long')

        elif z_score >= entry_threshold:
            # Short the spread (sell stock2, buy stock1)
            self._open_position(stock1, stock2, z_score, position_type='short')

    def _open_position(self, stock1, stock2, z_score, position_type):
        """
        Open a new pairs position.

        Parameters:
        -----------
        stock1, stock2 : str
            Stock symbols
        z_score : float
            Current z-score of the spread
        position_type : str
            'long' or 'short'
        """
        try:
            # Get account value for position sizing
            account = self.api.get_account()
            portfolio_value = float(account.equity)
            position_value = portfolio_value * self.position_size

            # Get current prices
            stock1_price = float(self.api.get_latest_trade(stock1).price)
            stock2_price = float(self.api.get_latest_trade(stock2).price)

            # Get hedge ratio
            hedge_ratio = self.pair_stats[(stock1, stock2)]['hedge_ratio']

            # Calculate quantities
            stock1_qty = int(position_value / (2 * stock1_price))
            stock2_qty = int(hedge_ratio * stock1_qty)

            # Ensure minimum quantities
            stock1_qty = max(stock1_qty, 1)
            stock2_qty = max(stock2_qty, 1)

            if position_type == 'long':
                # Long the spread: Buy stock2, Sell stock1
                self.log(f"Opening LONG spread position for {stock1}-{stock2} at Z-Score {z_score:.4f}")
                self.log(f"Selling {stock1_qty} shares of {stock1} and buying {stock2_qty} shares of {stock2}")

                self.api.submit_order(
                    symbol=stock1,
                    qty=stock1_qty,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )

                self.api.submit_order(
                    symbol=stock2,
                    qty=stock2_qty,
                    side='buy',
                    type='market',
                    time_in_force='gtc'
                )

            else:  # position_type == 'short'
                # Short the spread: Sell stock2, Buy stock1
                self.log(f"Opening SHORT spread position for {stock1}-{stock2} at Z-Score {z_score:.4f}")
                self.log(f"Buying {stock1_qty} shares of {stock1} and selling {stock2_qty} shares of {stock2}")

                self.api.submit_order(
                    symbol=stock1,
                    qty=stock1_qty,
                    side='buy',
                    type='market',
                    time_in_force='gtc'
                )

                self.api.submit_order(
                    symbol=stock2,
                    qty=stock2_qty,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )

            # Record the position details
            self.active_pairs[(stock1, stock2)] = {
                'position_type': position_type,
                'entry_zscore': z_score,
                'entry_time': datetime.now(),
                'stock1_qty': stock1_qty,
                'stock2_qty': stock2_qty,
                'stock1_price': stock1_price,
                'stock2_price': stock2_price
            }

        except Exception as e:
            self.log(f"Error opening position for pair {stock1}-{stock2}: {e}")

    def _close_position(self, stock1, stock2, z_score):
        """
        Close an existing pairs position.

        Parameters:
        -----------
        stock1, stock2 : str
            Stock symbols
        z_score : float
            Current z-score of the spread
        """
        try:
            if (stock1, stock2) not in self.active_pairs:
                self.log(f"No active position found for pair {stock1}-{stock2}")
                return

            position_details = self.active_pairs[(stock1, stock2)]
            position_type = position_details['position_type']
            stock1_qty = position_details['stock1_qty']
            stock2_qty = position_details['stock2_qty']

            entry_time = position_details['entry_time']
            holding_period = datetime.now() - entry_time

            # Get current prices for P&L calculation
            stock1_entry_price = position_details['stock1_price']
            stock2_entry_price = position_details['stock2_price']
            stock1_current_price = float(self.api.get_latest_trade(stock1).price)
            stock2_current_price = float(self.api.get_latest_trade(stock2).price)

            # Calculate P&L
            if position_type == 'long':
                # Long spread: Sold stock1, Bought stock2
                stock1_pnl = (stock1_entry_price - stock1_current_price) * stock1_qty
                stock2_pnl = (stock2_current_price - stock2_entry_price) * stock2_qty

                # Close the position: Buy back stock1, Sell stock2
                self.log(f"Closing LONG spread position for {stock1}-{stock2} at Z-Score {z_score:.4f}")
                self.log(f"Buying {stock1_qty} shares of {stock1} and selling {stock2_qty} shares of {stock2}")

                self.api.submit_order(
                    symbol=stock1,
                    qty=stock1_qty,
                    side='buy',
                    type='market',
                    time_in_force='gtc'
                )

                self.api.submit_order(
                    symbol=stock2,
                    qty=stock2_qty,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )

            else:  # position_type == 'short'
                # Short spread: Bought stock1, Sold stock2
                stock1_pnl = (stock1_current_price - stock1_entry_price) * stock1_qty
                stock2_pnl = (stock2_entry_price - stock2_current_price) * stock2_qty

                # Close the position: Sell stock1, Buy back stock2
                self.log(f"Closing SHORT spread position for {stock1}-{stock2} at Z-Score {z_score:.4f}")
                self.log(f"Selling {stock1_qty} shares of {stock1} and buying {stock2_qty} shares of {stock2}")

                self.api.submit_order(
                    symbol=stock1,
                    qty=stock1_qty,
                    side='sell',
                    type='market',
                    time_in_force='gtc'
                )

                self.api.submit_order(
                    symbol=stock2,
                    qty=stock2_qty,
                    side='buy',
                    type='market',
                    time_in_force='gtc'
                )

            total_pnl = stock1_pnl + stock2_pnl

            # Log the closed position
            self.log(f"Closed {position_type.upper()} spread position for {stock1}-{stock2}")
            self.log(f"Holding period: {holding_period}")
            self.log(f"P&L: ${total_pnl:.2f} (${stock1_pnl:.2f} from {stock1}, ${stock2_pnl:.2f} from {stock2})")

            # Remove from active pairs
            del self.active_pairs[(stock1, stock2)]

        except Exception as e:
            self.log(f"Error closing position for pair {stock1}-{stock2}: {e}")

    def run_backtest_on_top_pair(self, pairs, price_data, initial_capital=100000):
        """
        Run backtest on the top selected pair.
        
        Parameters:
        - pairs: list of tuples like (stock1, stock2, p_value)
        - price_data: dict of stock price series
        - initial_capital: capital for backtesting
        """
        if not pairs:
            self.log("No pairs to backtest.")
            return
        
        stock1, stock2, _ = pairs[0]
        thresholds = self.calculate_adaptive_thresholds(price_data, stock1, stock2)
        result = backtest_pair(price_data, stock1, stock2, thresholds, initial_capital=initial_capital)
    
        self.log(f"Backtest complete for top pair {stock1}-{stock2}")
        self.log(f"Final capital: ${result['summary']['Final Capital']:.2f}")
        self.log(f"Total return: {result['summary']['Total Return (%)']:.2f}%")
        
        return result

    def test(self, universe=None):
        
        if universe is None:
            # Default to S&P 500 stocks or another list
            universe = self._get_default_universe()
        return self.find_pairs(universe)
        
    
    def run(self, universe=None, interval=15):
        """
        Run the pairs trading strategy.

        Parameters:
        -----------
        universe : list
            List of stock symbols to analyze. If None, will use a default universe.
        interval : int
            Interval in minutes to check for signals
        """
        if universe is None:
            # Default to S&P 500 stocks or another list
            universe = self._get_default_universe()

        self.log(f"Starting pairs trading strategy with {len(universe)} stocks...")

        # Find initial pairs
        pairs_list, price_data = self.find_pairs(universe)
        self.log(f"Chosen pairs: {pairs_list}")
        

        #Main trading loop
        while True:
            try:
                # Check if market is open
                clock = self.api.get_clock()
                if clock.is_open:
                    self.log("Market is open. Checking for signals...")

                    # Check for trading signals
                    self.check_for_trading_signals()

                    # Wait for next interval
                    self.log(f"Waiting {interval} minutes until next check...")
                    time.sleep(interval * 60)
                else:
                    # Market closed, calculate time until next open
                    next_open = clock.next_open.timestamp() - time.time()
                    next_open_hours = next_open / 3600
                    self.log(f"Market is closed. Next opening in {next_open_hours:.2f} hours.")

                    # If it's near market open (< 1 hour), wait and then find new pairs
                    if next_open < 3600:
                        self.log("Less than 1 hour until market open. Waiting...")
                        time.sleep(next_open)

                        # Refresh pairs before market open
                        self.log("Finding new pairs before market open...")
                        pairs_list = self.find_pairs(universe)
                    else:
                        # Otherwise, sleep for an hour and check again
                        self.log("Sleeping for 1 hour...")
                        time.sleep(3600)

            except KeyboardInterrupt:
                self.log("Strategy interrupted by user.")
                break

            except Exception as e:
                self.log(f"Error in main loop: {e}")
                time.sleep(300)  # Wait 5 minutes before retrying

    def _get_default_universe(self):
        """Get a default universe of stocks to trade."""
        # This is a sample of liquid S&P 500 stocks
        return [
            "AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "JPM", "JNJ", "V",
            "PG", "UNH", "HD", "MA", "BAC", "DIS", "ADBE", "CRM", "NFLX", "PYPL",
            "INTC", "VZ", "CMCSA", "PEP", "ABT", "KO", "MRK", "PFE", "WMT", "T",
            "CSCO", "CVX", "XOM", "NKE", "ABBV", "TMO", "AVGO", "MCD", "ACN", "COST",
            "DHR", "LLY", "MDT", "TXN", "QCOM", "BMY", "UNP", "NEE", "MS", "C"
        ]




# Example usage (to be integrated into main.py)


In [16]:
# Integration with the main script
def run_pairs_trading(api, universe=None):
    """
    Run the pairs trading strategy.

    Parameters:
    -----------
    api : alpaca_trade_api.REST
        The Alpaca API client
    universe : list
        List of stock symbols to trade. If None, a default universe will be used.
    """
    # Initialize the pairs trader
    pairs_trader = PairsTrader(
        api=api,
        lookback_period=60,  # 60 days of historical data
        z_score_threshold=2.0,  # Open positions when z-score exceeds 2
        max_active_pairs=5,  # Trade at most 5 pairs simultaneously
        position_size=0.05  # Use 5% of portfolio per pair
    )

    # Run the strategy
    pairs_trader.run(universe=universe, interval=15)
    # pairs_list, price_data = pairs_trader.test()
    # pairs_trader.run_backtest_on_top_pair(pairs_list, price_data)

    


In [ ]:
if __name__ == "__main__":
    # Load config and initialize API
    def load_config(config_path='untitled.ini'):
        config = configparser.ConfigParser()
        config.read(config_path)
        return {
            'api_key': config['alpaca']['api_key'],
            'api_secret': config['alpaca']['api_secret'],
            'base_url': 'https://paper-api.alpaca.markets'
        }


    def init_api(config):
        return REST(config['api_key'], config['api_secret'], config['base_url'])


    config = load_config()
    api = init_api(config)

    # Run the pairs trading strategy
    run_pairs_trading(api)

[2025-04-24 14:42:20] Starting pairs trading strategy with 50 stocks...
[2025-04-24 14:42:20] Finding cointegrated pairs from 50 stocks...
[2025-04-24 14:42:25] Available stocks in price_data: ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'JPM', 'JNJ', 'V', 'PG', 'UNH', 'HD', 'MA', 'BAC', 'DIS', 'ADBE', 'CRM', 'NFLX', 'PYPL', 'INTC', 'VZ', 'CMCSA', 'PEP', 'ABT', 'KO', 'MRK', 'PFE', 'WMT', 'T', 'CSCO', 'CVX', 'XOM', 'NKE', 'ABBV', 'TMO', 'AVGO', 'MCD', 'ACN', 'COST', 'DHR', 'LLY', 'MDT', 'TXN', 'QCOM', 'BMY', 'UNP', 'NEE', 'MS', 'C']
[2025-04-24 14:42:25] Using sector/industry analysis to find pairs...
Processing batch 1/3...


  0%|          | 0/20 [00:00<?, ?it/s]

Processing batch 2/3...


  0%|          | 0/20 [00:00<?, ?it/s]

Processing batch 3/3...


  0%|          | 0/10 [00:00<?, ?it/s]

Processing industry: Internet Content & Information (2 stocks)
Processing industry: Semiconductors (5 stocks)
Processing industry: Banks - Diversified (3 stocks)
Processing industry: Drug Manufacturers - General (6 stocks)
Processing industry: Credit Services (3 stocks)
Processing industry: Entertainment (2 stocks)
Processing industry: Software - Application (2 stocks)
Processing industry: Telecom Services (3 stocks)
Processing industry: Beverages - Non-Alcoholic (2 stocks)
Processing industry: Medical Devices (2 stocks)
Processing industry: Discount Stores (2 stocks)
Processing industry: Oil & Gas Integrated (2 stocks)
Processing industry: Diagnostics & Research (2 stocks)
Processing sector: Technology (11 stocks)
Processing sector: Consumer Cyclical (5 stocks)
Processing sector: Communication Services (7 stocks)
Processing sector: Financial Services (7 stocks)
Processing sector: Healthcare (11 stocks)
Processing sector: Consumer Defensive (5 stocks)
Processing sector: Energy (2 stock

C:\Users\DHEERAJ KRISHNA\AppData\Local\Temp\ipykernel_28300\3782891574.py:300: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  current = float(spread[-1])


[2025-04-24 14:42:42] Saved plot for MSFT-ADBE to pair_visualizations\MSFT_ADBE_visualization.png
Spread Mean: -479.3780
Spread Std Dev: 25.6333
Current Spread: -499.2806
Current Z-Score: -0.7764
[2025-04-24 14:42:42] Saved plot for MSFT-CRM to pair_visualizations\MSFT_CRM_visualization.png
Spread Mean: -254.3871
Spread Std Dev: 9.2963
Current Spread: -263.4011
Current Z-Score: -0.9696
[2025-04-24 14:42:43] Saved plot for PFE-TMO to pair_visualizations\PFE_TMO_visualization.png
Spread Mean: -55.7694
Spread Std Dev: 7.0348
Current Spread: -63.0477
Current Z-Score: -1.0346
[2025-04-24 14:42:44] Saved plot for JNJ-DHR to pair_visualizations\JNJ_DHR_visualization.png
Spread Mean: -66.0197
Spread Std Dev: 5.6038
Current Spread: -63.0598
Current Z-Score: 0.5282
[2025-04-24 14:42:44] Saved plot for MA-MS to pair_visualizations\MA_MS_visualization.png
Spread Mean: -69.1248
Spread Std Dev: 3.1248
Current Spread: -71.9474
Current Z-Score: -0.9033
[2025-04-24 14:42:45] Saved plot for CSCO-TXN to 